Day 9: PostgreSQL with SQLAlchemy

Added dotenv functionality

In [3]:
from sqlalchemy import create_engine, text
import pandas as pd
import os
from dotenv import load_dotenv

class DatabaseConnection:
    """Professional database connection class"""
    
    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.engine = None
    
    def connect(self):
        """Create database engine"""
        try:
            self.engine = create_engine(self.connection_string)
            print("Database connection established")
            return True
        except Exception as e:
            print(f"Connection error: {e}")
            return False
    
    def execute_query(self, query, params=None):
        """Execute query and return DataFrame"""
        try:
            if params:
                df = pd.read_sql_query(text(query), self.engine, params=params)
            else:
                df = pd.read_sql_query(text(query), self.engine)
            return df
        except Exception as e:
            print(f"Query error: {e}")
            return None
    
    def close(self):
        """Close database connection"""
        if self.engine:
            self.engine.dispose()
            print("Database connection closed")

# Load environment variables
load_dotenv()

# Get database URL from .env file
database_url = os.getenv('DATABASE_URL')

# Create database connection instance
db = DatabaseConnection(database_url)

# Connect and execute queries
if db.connect():
    # Test connection first
    test_query = "SELECT version();"
    test_df = db.execute_query(test_query)
    if test_df is not None:
        print("Connection test successful!")
        print(test_df)
    
    # Your practice query (adjusted for Chinook database)
    query = """
    SELECT 
        il.invoice_line_id,
        i.invoice_id,
        i.invoice_date,
        t.name as track_name,
        a.title as album_title,
        ar.name as artist_name,
        g.name as genre_name,
        il.unit_price,
        il.quantity
    FROM invoice_line il 
    LEFT JOIN invoice i ON il.invoice_id = i.invoice_id
    INNER JOIN track t ON il.track_id = t.track_id
    INNER JOIN album a ON t.album_id = a.album_id
    INNER JOIN artist ar ON a.artist_id = ar.artist_id
    INNER JOIN genre g ON t.genre_id = g.genre_id
    WHERE i.invoice_date > '2010-01-01'
    LIMIT 10;
    """
    
    df = db.execute_query(query)
    if df is not None:
        print("\nQuery Results:")
        print(df.head())
        print(f"\nTotal rows returned: {len(df)}")
    
    # Close connection
    db.close()
else:
    print("Failed to connect to database. Check your .env file and PostgreSQL setup.")

Database connection established
Connection test successful!
                                             version
0  PostgreSQL 17.5 on x86_64-windows, compiled by...

Query Results:
   invoice_line_id  invoice_id invoice_date  \
0              579         108   2022-04-13   
1                1           1   2021-01-01   
2             1154         214   2023-07-25   
3             1728         319   2024-11-01   
4                2           1   2021-01-01   

                                track_name  \
0  For Those About To Rock (We Salute You)   
1                        Balls to the Wall   
2                        Balls to the Wall   
3                          Fast As a Shark   
4                        Restless and Wild   

                             album_title artist_name genre_name  unit_price  \
0  For Those About To Rock We Salute You       AC/DC       Rock        0.99   
1                      Balls to the Wall      Accept       Rock        0.99   
2                    

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()
db_url = os.getenv('DATABASE_URL')